In [1]:
import time
import numpy as np
import pandas as pd
import ringity as rng
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from itertools import product
from collections import defaultdict

In [2]:
N = 2**9
rho = 0.05

In [3]:
def time_estimation(i, iter_len):
    TIME2 = time.time()
    t_cur = TIME2 - TIME1
    prog = (i+1)/iter_len
    t_rem = t_cur * (1/prog - 1)
    print(f"time passed: {t_cur:.2f}sec - progress: {100*prog:.2f}% - time remaining: {t_rem:.2f}",
          end = '\r')

In [6]:
scores = defaultdict(dict)
n_a_samples = 2**2
n_b_samples = 2**2
n_epochs = 2**2

a_arr = np.linspace(rho, 0.5, n_a_samples)
b_arr = np.linspace(0.8, 1, n_b_samples)

TIME1 = time.time()
count = 0
for a, beta in product(a_arr, b_arr):
    scores[a][beta] = []
    for epoch in range(n_epochs):
        count += 1
        time_estimation(count, n_a_samples*n_b_samples*n_epochs)
        try:
            G = rng.network_model(N=N, rho=rho, beta=beta, a=a)
            score = rng.diagram(G).ring_score() # deprication warning
        except (ValueError, rng.DisconnectedGraphError):
            score = np.nan
        
        scores[a][beta].append(score)

In [8]:
plot_df = pd.DataFrame(scores)

In [23]:
df_list = []
for a, score_dic in scores.items():
    df = pd.DataFrame(score_dic)
    df['a'] = a
    df_list.append(df)

In [51]:
df = (pd.
        concat(df_list).
        melt(id_vars='a', 
             value_name='score', 
             var_name='beta'))

In [54]:
plot_df = (df.
           groupby(['beta', 'a']).
           mean().
           reset_index())

In [55]:
plot_df

,beta,a,score
0,0.800000,0.05,0.956505
1,0.800000,0.20,0.316471
2,0.800000,0.35,0.038596
3,0.800000,0.50,0.030859
4,0.866667,0.05,0.969670
5,0.866667,0.20,0.495691
6,0.866667,0.35,0.022115
7,0.866667,0.50,0.013285
8,0.933333,0.05,0.965876
9,0.933333,0.20,0.636500


In [61]:
# Create figure
fig = go.Figure()

# Add traces, one for each slider step
for (beta, score_df) in plot_df.groupby('beta'):
    fig.add_trace(
        go.Scatter(
            visible = False,
            line = dict(color="#00CED1", width=6),
            name = fr"$\beta$ = {beta:.2f}",
            x = score_df.a,
            y = score_df.score
        )
    )
    
    fig.update_yaxes(range = [-0.05, 1.05])

# Make 10th trace visible
fig.data[-1].visible = True

# Create and add slider
steps = []
for i, (beta, _) in enumerate(plot_df.groupby('beta')):
    step = dict(
        method = "update",
        args = [{"visible": [False] * len(fig.data)},
                {"title": f"Slider switched to beta: {beta:.2f}"}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Frequency: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()

In [65]:
plot_df.sample(3)

,beta,a,score
4,0.866667,0.05,0.969670
11,0.933333,0.50,0.028131
9,0.933333,0.20,0.636500
